In [1]:
import numpy as np
import pandas as pd
from scipy import signal
import os,obspy,shutil,glob
from obspy import read
import librosa.display 
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['agg.path.chunksize'] = 10000

from utils import *
from utils.utils import *
from utils import data_process, visualize
# from utils.models import Att_CNN, CNN, CRNN,M_CRNN
# from utils.TFEQ import TFEQ, TFEQ_v1, TFEQ_v2

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "7"

###======Settings for different test cases======###
SamplingRate = 100 # need to be changed, 25/50/100
Duration = 2 # need to be changed, 2/4/10
###============###
WindowSize = 2 * SamplingRate
original_SamplingRate = 100
rate = original_SamplingRate/SamplingRate

In [2]:
def pro_data(ex,ey,ez):
    X_EQ, Y_EQ, Z_EQ = [], [], []
    X_tg, Y_tg, Z_tg =ex,ey,ez

    ## 2 sec sliding window with 1 sec overlap
    for j in np.arange(0, len(X_tg) - SamplingRate, SamplingRate):
        X_batch = X_tg[j:j + WindowSize]
        Y_batch = Y_tg[j:j + WindowSize]
        Z_batch = Z_tg[j:j + WindowSize]
        if len(X_batch) == WindowSize:
            X_EQ.append(X_batch)
            Y_EQ.append(Y_batch)
            Z_EQ.append(Z_batch)

    X_EQ, Y_EQ, Z_EQ = np.asarray(X_EQ), np.asarray(Y_EQ), np.asarray(Z_EQ)

    X_EQ = X_EQ.reshape(int(len(X_EQ) / (Duration - 1)), Duration - 1, WindowSize, 1)
    Y_EQ = Y_EQ.reshape(int(len(Y_EQ) / (Duration - 1)), Duration - 1, WindowSize, 1)
    Z_EQ = Z_EQ.reshape(int(len(Z_EQ) / (Duration - 1)), Duration - 1, WindowSize, 1)

    X_EQ = X_EQ.reshape(X_EQ.shape[0]*X_EQ.shape[1], X_EQ.shape[2], 1)
    Y_EQ = Y_EQ.reshape(Y_EQ.shape[0]*Y_EQ.shape[1], Y_EQ.shape[2], 1)
    Z_EQ = Z_EQ.reshape(Z_EQ.shape[0]*Z_EQ.shape[1], Z_EQ.shape[2], 1)

    EQ_X = np.dstack((X_EQ, Y_EQ, Z_EQ))
    x_train = torch.from_numpy(EQ_X).float()
    return x_train

filepath='/home/yaso/EQ_Place/dataset/New/20220416/'
qry = filepath +'*.mseed'
files = glob.glob(qry)
files.sort()
data, x_train=[],[]
for i in tqdm(range(len(files))):
    st = read(files[i])

    ex, ey, ez = st[0].data, st[1].data, st[2].data

    ex  = [x * (2.5 / 2**15) for x in ex]
    ey = [x * (2.5 / 2**15) for x in ey] 
    ez =[x * (2.5 / 2**15) for x in ez]
    ex, ey, ez = ex - np.mean(ex), ey - np.mean(ey), ez - np.mean(ez)
    data.append([ex,ey,ez])
    x_train.append(pro_data(ex,ey,ez))

100%|██████████| 101/101 [00:49<00:00,  2.04it/s]


In [15]:
# filepath='/home/yaso/EQ_Place/dataset/EQ_20210829_130323/'
# qry = filepath +'01232972597.csv'
# files = glob.glob(qry)
# data, x_train=[],[]
# for i in tqdm(range(len(files))):
#     st = pd.read_csv(files[i])
#     st=np.array(st)
#     ex, ey, ez = st[:,0], st[:,1], st[:,2]
#     ex, ey, ez = ex - np.mean(ex), ey - np.mean(ey), ez - np.mean(ez)
#     data.append([ex,ey,ez])
#     x_train.append(pro_data(ex,ey,ez))

In [3]:
class Conv_com(nn.Module):
    def __init__(self, ):
        super(Conv_com, self).__init__()
        self.conv_1 = nn.Conv1d(
            in_channels=1,  out_channels=32, kernel_size=3, stride=2)
        self.conv_2 = nn.Conv1d(
            in_channels=32, out_channels=32, kernel_size=3, stride=2)
        self.conv_3 = nn.Conv1d(in_channels=32, out_channels=32, kernel_size=3)
        self.MaxPool = nn.MaxPool1d(2)
        self.rnn = nn.RNN(input_size=736, hidden_size=100)
        self.fla = torch.nn.Flatten()
        self.fc1 = nn.Linear(100, 2)

    def forward(self, x):
        out = F.relu(self.conv_1(x))
        out = F.relu(self.conv_2(out))
        out = F.relu(self.conv_3(out))
        out = self.MaxPool(out)
        out = self.fla(out)
        out = out.view(-1, 1, out.shape[-1])
        out, _ = self.rnn(out)
        out = self.fla(out)
        return out    

class M_CRNN(nn.Module):
    def __init__(self, ):
        super(M_CRNN, self).__init__()
        self.conv_x = Conv_com()
        self.conv_y = Conv_com()
        self.conv_z = Conv_com()
        self.rnn = nn.RNN(input_size=2208, hidden_size=100)
        self.fla = nn.Flatten()
        self.dp = nn.Dropout(p=0.5)
        self.fc1 = nn.Linear(300, 100)
        self.fc2 = nn.Linear(100, 2)

    def forward(self, data):
        x, y, z = data[:, 0, :], data[:, 1, :], data[:, 2, :]
        x = x.view(-1, 1, x.shape[-1])
        y = y.view(-1, 1, y.shape[-1])
        z = z.view(-1, 1, z.shape[-1])
        x_out = self.conv_x(x)
        y_out = self.conv_y(y)
        z_out = self.conv_z(z)
        new_feature = torch.cat([x_out, y_out, z_out], dim=1)
#         new_feature = new_feature.view(-1, 1, new_feature.shape[-1])
#         out, _ = self.rnn(new_feature)
        out = F.relu(self.fc1(new_feature))
        out = self.dp(out)
#         out = F.relu(self.fc1(out))
        out = self.fc2(out)
        return F.log_softmax(out, dim=1)

In [4]:
m1=torch.load('./models/TFEQ_v2.pth')
m1.eval()
enable_dropout(m1)

os.makedirs('./figures/TFEQ_v2/', exist_ok=True)


mc_num=10

In [5]:
def draw_predict_mc(data, probability, show=False, save_path=None, save=False, lim=0.01,mc_num=10, yline=0.0025):
    x_lim = np.arange(0, len(data[0]), 1)
    fig, axs = plt.subplots(3, 1, sharex=False, figsize=(6, 5))

    axs[0].axhline(y=yline, c="green",  linewidth=0.8)
    axs[0].axhline(y=-yline, c="green",  linewidth=0.8)
    axs[0].plot(x_lim, data[0], linewidth=0.5)
    axs[0].plot(x_lim, data[1], linewidth=0.5)
    axs[0].plot(x_lim, data[2], linewidth=0.5)
    axs[0].set_ylim((-lim, lim))
    axs[0].grid(True)

    x_lim = np.arange(0, len(probability[0]), 1)
    for m in range(mc_num):
        axs[1].plot(x_lim, probability[m], linewidth=0.5)
    axs[1].set_ylim((0, 1))
    axs[1].grid(True)
    
    axs[2].plot(x_lim, probability.mean(axis=0), linewidth=0.8)
    axs[2].set_ylim((0, 1))
    axs[2].grid(True)
    plt.tight_layout()
    if save:
        plt.savefig(save_path, dpi=100)
    if not show:
        plt.clf()
        plt.close()

In [14]:
for i in tqdm(range(25, len(files))):
    d1=x_train[i][:300].cuda()
    d2=x_train[i][300:400].cuda()
    d3=x_train[i][400:].cuda()
    probability=[]
    for m in range(mc_num):
        output = m1(d1)
        output = F.softmax(output, dim=1)[:,1]
        p1=output.squeeze().cpu().detach().numpy()

        output = m1(d2)
        output = F.softmax(output, dim=1)[:,1]
        p2=output.squeeze().cpu().detach().numpy()
        
        output = m1(d3)
        output = F.softmax(output, dim=1)[:,1]
        p3=output.squeeze().cpu().detach().numpy()
        
        p=np.hstack([p1,p2,p3])
        probability.append(p)
    probability=np.vstack(probability)
    draw_predict_mc(data[i], probability, show=False,save=True,mc_num=10,save_path='figures/TFEQ_v2/'+files[i][-17:-6])

100%|██████████| 76/76 [01:04<00:00,  1.18it/s]


In [13]:
i

24

In [3]:
m1=torch.load('./models/TFEQ.pth')
m1.eval()
enable_dropout(m1)
m2=torch.load('./models/CRNN.pth')
m2.eval()
enable_dropout(m2)
m3=torch.load('./models/M_CRNN.pth')
m3.eval()
enable_dropout(m3)

os.makedirs('./figures/TFEQ/', exist_ok=True)
os.makedirs('./figures/CRNN/', exist_ok=True)
os.makedirs('./figures/M_CRNN/', exist_ok=True)

mc_num=10

In [7]:
def draw_predict_mc(data, probability, show=False, save_path=None, save=False, lim=0.01,mc_num=10, yline=0.0025):
    x_lim = np.arange(0, len(data[0]), 1)
    fig, axs = plt.subplots(3, 1, sharex=False, figsize=(6, 5))

    axs[0].axhline(y=yline, c="green",  linewidth=0.8)
    axs[0].axhline(y=-yline, c="green",  linewidth=0.8)
    axs[0].plot(x_lim, data[0], linewidth=0.5)
    axs[0].plot(x_lim, data[1], linewidth=0.5)
    axs[0].plot(x_lim, data[2], linewidth=0.5)
    axs[0].set_ylim((-lim, lim))
    axs[0].grid(True)

    x_lim = np.arange(0, len(probability[0]), 1)
    for m in range(mc_num):
        axs[1].plot(x_lim, probability[m], linewidth=0.5)
    axs[1].set_ylim((0, 1))
    axs[1].grid(True)
    
    axs[2].plot(x_lim, probability.mean(axis=0), linewidth=0.8)
    axs[2].set_ylim((0, 1))
    axs[2].grid(True)
    plt.tight_layout()
    if save:
        plt.savefig(save_path, dpi=100)
    if not show:
        plt.clf()
        plt.close()

In [5]:
for i in tqdm(range(len(files))):
    d1=x_train[i][:300].cuda()
    d2=x_train[i][300:600].cuda()
    d3=x_train[i][600:].cuda()
    probability=[]
    for m in range(mc_num):
        output = m1(d1)
        output = F.softmax(output, dim=1)[:,1]
        p1=output.squeeze().cpu().detach().numpy()

        output = m1(d2)
        output = F.softmax(output, dim=1)[:,1]
        p2=output.squeeze().cpu().detach().numpy()
        
        output = m1(d3)
        output = F.softmax(output, dim=1)[:,1]
        p3=output.squeeze().cpu().detach().numpy()
        
        p=np.hstack([p1,p2,p3])
        probability.append(p)
    probability=np.vstack(probability)
    draw_predict_mc(data[i], probability, show=False,save=True,mc_num=10,save_path='figures/TFEQ/'+files[i][-17:-6])
#     break

100%|██████████| 101/101 [01:16<00:00,  1.32it/s]


In [ ]:
for i in tqdm(range(len(files))):
    d=x_train[i].reshape(-1, 3, 200).cuda()
    probability=[]
    for m in range(mc_num):
        output = m3(d)
        output = F.softmax(output, dim=1)[:,1]
        p=output.squeeze().cpu().detach().numpy()
        probability.append(p)
    probability=np.vstack(probability)
#     draw_predict(data[i], probability, show=True)
    draw_predict_mc(data[i], probability, show=False,save=True,save_path='figures/M_CRNN/'+files[i][-17:-6])

In [17]:
# no-Mc dropout
for i in tqdm(range(len(files))):
    d=x_train[i][:420].cuda()
    output = m1(d)
    output = F.softmax(output, dim=1)[:,1]
    probability=output.squeeze().cpu().detach().numpy()
    d=x_train[i][420:].cuda()
    output = m1(d)
    output = F.softmax(output, dim=1)[:,1]
    probability1=output.squeeze().cpu().detach().numpy()

    probability=np.hstack([probability,probability1])
    draw_predict(data[i], probability, show=False,save=True,save_path='figures/TFEQ/'+files[i][-17:-6])

100%|██████████| 1/1 [00:00<00:00,  2.28it/s]


In [18]:
# no-Mc dropout
for i in tqdm(range(len(files))):
    d=x_train[i].reshape(-1, 3, 200).cuda()
    output = m2(d)
    output = F.softmax(output, dim=1)[:,1]
    probability=output.squeeze().cpu().detach().numpy()
#     draw_predict(data[i], probability, show=True)
    draw_predict(data[i], probability, show=False,save=True,save_path='figures/CRNN/'+files[i][-17:-6])

100%|██████████| 1/1 [00:00<00:00,  2.38it/s]


In [19]:
# no-Mc dropout
for i in tqdm(range(len(files))):
    d=x_train[i].reshape(-1, 3, 200).cuda()
    output = m3(d)
    output = F.softmax(output, dim=1)[:,1]
    probability=output.squeeze().cpu().detach().numpy()
#     draw_predict(data[i], probability, show=True)
    draw_predict(data[i], probability, show=False,save=True,save_path='figures/M_CRNN/'+files[i][-17:-6])

100%|██████████| 1/1 [00:00<00:00,  1.85it/s]


In [18]:
class CRNN(nn.Module):
    def __init__(self, ):
        super(CRNN, self).__init__()
        self.conv1 = nn.Conv1d(
            in_channels=3,  out_channels=64, kernel_size=3, stride=2)
        self.conv2 = nn.Conv1d(
            in_channels=64,  out_channels=64, kernel_size=3, stride=1)
        self.pool1 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.fla = nn.Flatten()
        self.dp = nn.Dropout(p=0.1)
        self.rnn = nn.RNN(input_size=3072, hidden_size=100)
        self.fc1 = nn.Linear(100, 2)

    def forward(self, data):
        out = F.relu(self.conv1(data))
        out = F.relu(self.conv2(out))
        out = self.dp(out)
        out = self.pool1(out)
        out = self.fla(out)
        out = out.view(-1, 1, out.shape[-1])
        out, _ = self.rnn(out)
        out = self.fla(out)
        out = self.fc1(out)
        return F.log_softmax(out, dim=1)

m2=torch.load('./models/CRNN_d1.pth')
m2.eval()
enable_dropout(m2)

os.makedirs('./figures/CRNN_d1/', exist_ok=True)
mc_num=10

for i in tqdm(range(len(files))):
    d=x_train[i].reshape(-1, 3, 200).cuda()
    probability=[]
    for m in range(mc_num):
        output = m2(d)
        output = F.softmax(output, dim=1)[:,1]
        p=output.squeeze().cpu().detach().numpy()
        probability.append(p)
    probability=np.vstack(probability)
#     draw_predict(data[i], probability, show=True)
    draw_predict_mc(data[i], probability, show=False,save=True,save_path='figures/CRNN_d1/'+files[i][-17:-6])

01232972140, 01232972597, 01232971750, 01232973198

01232973617  